# Welcome to Knowledge Check 3! 

Today we're going to combine some data.

## Combining Data

Data taken from [Louisville Metro Data](https://data.louisvilleky.gov/dataset/parking-citations). This is the same data from Knowledge Check 2, but we're going to pull in a table of codes to try to see what these parking tickets mean. At the bottom of the page, you can see a csv titled ```Violation Codes``` - it's fairly common to see a data set up like this, where you have the codes in one table (or file) and the description of them in another. For exampe, at Humana, we have a table that has CPT Code information, like descriptions of the type of surgery, and then on the claims table we just have the CPT code (this is just a code for what surgery was performed). I routinely have to just join these together and pull the codes and claims from one table and descriptions from another.

---

1. Make a .py (or .ipynb) file that contains the following (your choice of editor does not matter!) and do the following: 
- import two sets of data from any source you want. They can be typed by hand like I did below, imported from an API, or any other way.
- join them / combine them into one data frame. You can use any kind of join you want, but I find inner joins and left joins are the most common by far. Play around with different types though if you want to see what they do.
2. Commit your changes.
3. Push your changes to your repo and notify your mentor!

In [1]:
import pandas as pd 

## Code Examples

For the sake of simplicity, I'm going to type some data in and join the two sets together. The important things to note: 
- the 'key' I'm joining on is ```country```, and the names are exactly identical. If one had "USA" and one had "United States", the join wouldn't work. It's usually much easier to join on, say, a number like a Tax ID or a unique identifying number rather than a string for the reasons mentioned above.

In [ ]:
left_table = [
    {"country" : "USA",
    "population" : 330000000},
    {"country" : "Argentina",
     "population" : 45800000},
     {"country" : "Brazil",
      "population" : 212300000}
]

right_table = [
    {"country" : "USA",
    "GDP" : 20000000000000},
    {"country" : "Argentina",
     "GDP" : 383000000000},
     {"country" : "Brazil",
      "GDP" : 1145000000000}
]

In [ ]:
pop_df = pd.DataFrame(data=left_table)
pop_df.head()

,country,population
0,USA,330000000
1,Argentina,45800000
2,Brazil,212300000


In [ ]:
gdp_df = pd.DataFrame(data=right_table)
gdp_df.head()

,country,GDP
0,USA,20000000000000
1,Argentina,383000000000
2,Brazil,1145000000000


Pandas understands that because I'm using the join method of pop_df, that that will be the left table and gdp_df is the right. It tries to match the key automatically based on the names of the columns that are present in both tables. However, you can specify which key from the left table and which from the right you want to join on. There's also an optional ```how``` argument to specify what type of join you want. 

The docs for DataFrame.merge are [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html).

Now we can do a calculation with both columns, hence our desire to combine them.

In [ ]:
combined_df = pop_df.merge(gdp_df)
combined_df['GDP_per_population'] = combined_df['GDP'] / combined_df['population']
combined_df.head()

,country,population,GDP,GDP_per_population
0,USA,330000000,20000000000000,60606.060606
1,Argentina,45800000,383000000000,8362.445415
2,Brazil,212300000,1145000000000,5393.311352


Let's try the same process with some real data.

In [2]:
citations = pd.read_csv('https://raw.githubusercontent.com/WillTirone/code_lou_work/main/Parking%20Citations%202018%20Thru%203-24-18.csv')
codes = pd.read_csv('https://raw.githubusercontent.com/WillTirone/code_lou_work/main/Violation%20Codes.csv')

In [3]:
print(citations.columns)
print(codes.columns)

Index(['Cite Number', 'Issue Date', 'Violat', 'Sublocatio', 'Street',
       'Meter #', 'Is Wa', 'Due'],
      dtype='object')
Index(['Code', 'Description', 'Dollar Fine'], dtype='object')


In [4]:
citations.head()

,Cite Number,Issue Date,Violat,Sublocatio,Street,Meter #,Is Wa,Due
0,M201507360,02/16/2018,PSW,700,YORK ST,NaN,NO,0.0
1,M201306003,01/05/2018,HO,100S,1ST ST,NaN,NO,0.0
2,M202705546,01/18/2018,X,N200,1ST ST,3012,NO,0.0
3,M202802800,01/10/2018,NPP,500W,LIBERTY ST,NaN,NO,0.0
4,M200208507,02/23/2018,NP,200E,CHESTNUT ST,NaN,NO,0.0


In [ ]:
codes.head()

,Code,Description,Dollar Fine
0,H,72.004 HANDICAP ZONE,100
1,RV,72.031 NO PARK RV/TRAILER,15
2,NPU,72.031 NO PARK TRAILER,15
3,U,72.032 UNATTENDED VEHICLE,15
4,PCV,72.033 NO PARK COMM VEH,15


In [ ]:
parking_df = citations.merge(codes,
                            how='left',
                            left_on='Violat',
                            right_on='Code')
parking_df.head()

,Cite Number,Issue Date,Violat,Sublocatio,Street,Meter #,Is Wa,Due,Code,Description,Dollar Fine
0,M201507360,02/16/2018,PSW,700,YORK ST,NaN,NO,0.0,PSW,72.035(C)NO PARK SIDEWALK,15.0
1,M201306003,01/05/2018,HO,100S,1ST ST,NaN,NO,0.0,NaN,NaN,NaN
2,M202705546,01/18/2018,X,N200,1ST ST,3012,NO,0.0,X,72.083(A)EXPIRED METER,15.0
3,M202802800,01/10/2018,NPP,500W,LIBERTY ST,NaN,NO,0.0,NPP,72.038(B)(7)NO STOP/PEAK PERIOD,15.0
4,M200208507,02/23/2018,NP,200E,CHESTNUT ST,NaN,NO,0.0,NP,72.035 NO PARKING,15.0


Nice! We combined the datasets to find the Description and Dollar Fine for each of the parking violations. Notice how I had to specify that I wanted to use a ```left_on``` and ```right_on``` argument for the join because the columns have different names. We could have renamed them (and like I mentioned last time, the columns of the first set have typos in them) but this is just for illustrations sake.

Notice below, though, that some of the Violations don't appear to have an even code mapping. For example, the violation ```HO``` just has a null value in the code column. This indicates that there isn't an equivalent in the code table (for whatever reason) so everything on the right side table is just filled in with nulls. If we used an inner join, that would have removed this row

In [ ]:
parking_df

,Cite Number,Issue Date,Violat,Sublocatio,Street,Meter #,Is Wa,Due,Code,Description,Dollar Fine
0,M201507360,02/16/2018,PSW,700,YORK ST,NaN,NO,0.0,PSW,72.035(C)NO PARK SIDEWALK,15.0
1,M201306003,01/05/2018,HO,100S,1ST ST,NaN,NO,0.0,NaN,NaN,NaN
2,M202705546,01/18/2018,X,N200,1ST ST,3012,NO,0.0,X,72.083(A)EXPIRED METER,15.0
3,M202802800,01/10/2018,NPP,500W,LIBERTY ST,NaN,NO,0.0,NPP,72.038(B)(7)NO STOP/PEAK PERIOD,15.0
4,M200208507,02/23/2018,NP,200E,CHESTNUT ST,NaN,NO,0.0,NP,72.035 NO PARKING,15.0
...,...,...,...,...,...,...,...,...,...,...,...
20331,M201900192,03/09/2018,OFF2,48,GARAGE - LOUISVILLE GARDENS,NaN,NO,135.0,NaN,NaN,NaN
20332,M200208453,01/16/2018,OFF3,W600,LOT - CITY HALL,NaN,NO,0.0,NaN,NaN,NaN
20333,M201610981,03/20/2018,OFF3,800S,LOT - MUD,NaN,NO,15.0,NaN,NaN,NaN
20334,M200706429,02/16/2018,NPP,300W,MARKET ST,NaN,NO,0.0,NPP,72.038(B)(7)NO STOP/PEAK PERIOD,15.0


In [ ]:
parking_df['Excess Fine'] = parking_df['Dollar Fine'] - parking_df['Due']

Now we can use this new data set. Since we have the fines from the code list, we can compare that to the ```Due``` column. If ```Due``` is greater than ```Dollar Fine``` that might indicate the customer was charged too much for the fine.

More likely, though, there's a late charge for not paying the fine in time, the fine table is inaccurate, or there's a surcharge because they've received several fines in the same time period. In any case, we have more questions! 

Here's the SQL equivalent of that join: 



```SQL
SELECT
  a.*,
  b.*
FROM citations a 
LEFT JOIN codes b 
ON a.Violat = b.Code
```



In [ ]:
parking_df[parking_df['Excess Fine'] < 0].head()

,Cite Number,Issue Date,Violat,Sublocatio,Street,Meter #,Is Wa,Due,Code,Description,Dollar Fine,Excess Fine
7,M201608912,01/02/2018,X,600W,LIBERTY ST,697,NO,50.0,X,72.083(A)EXPIRED METER,15.0,-35.0
9,M200706259,02/01/2018,X,700W,MAIN ST,305,NO,50.0,X,72.083(A)EXPIRED METER,15.0,-35.0
11,M200208520,02/23/2018,X,500S,FLOYD ST,FL521B,NO,50.0,X,72.083(A)EXPIRED METER,15.0,-35.0
12,M200610593,02/23/2018,NPP,400,MUHAMMAD ALI BLVD,NaN,NO,50.0,NPP,72.038(B)(7)NO STOP/PEAK PERIOD,15.0,-35.0
16,M201610531,03/06/2018,X,600S,BROOK ST,1141,NO,50.0,X,72.083(A)EXPIRED METER,15.0,-35.0
